# In this notebook some of the most fundamental concepts of tensors using TensorFlow are covered

More specifically:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercises


## Introduction to Tensors

In [1]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [ ]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [ ]:
# Check the number of dimensions
scalar.ndim

0

In [ ]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [ ]:
vector.ndim

1

In [ ]:
# Create a matrix
matrix = tf.constant([[10, 7],
                      [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [ ]:
matrix.ndim

2

In [ ]:
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the data type
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [ ]:
another_matrix.ndim

2

In [ ]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                       [[7, 8, 9],
                        [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [ ]:
tensor.ndim

3

### Creating tensors with `tf.Variable`

In [ ]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [ ]:
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [ ]:
# Let's try change one of the elements in our changeable tensor
# changeable_tensor[0] = 7
# changeable_tensor

In [ ]:
# Try .assign()
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# Let's try change our unchangeable tensor
# unchangeable_tensor[0].assign(7)
# unchangeable_tensor

### Creating random tensors

In [ ]:
# Create two random (but the same) tensors

random_1 = tf.random.Generator.from_seed(7)
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3, 2))

# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-1.3240396 ,  0.28785667],
        [-0.8757901 , -0.08857018],
        [ 0.69211644,  0.84215707]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

In [ ]:
# Shuffle a tensor
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
not_shuffled.ndim

2

In [ ]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [ ]:
# Shuffle our non-shuffled tensor
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operational level random seed:
> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjunction to determine the random sequence".

### Other ways to make tensors

In [ ]:
# Create the tensor of all ones
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [ ]:
# Create the tensor of all zeros
tf.zeros(shape=(3, 4))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]], dtype=float32)>

### Turn NumPy arrays into tensors

The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing)

In [ ]:
import numpy as np

numpy_A = np.arange(1, 25, dtype=np.int32)
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [ ]:
A = tf.constant(numpy_A, shape=(3, 8))
B = tf.constant(numpy_A)
A, B

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16],
        [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>,
 <tf.Tensor: shape=(24,), dtype=int32, numpy=
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24], dtype=int32)>)

### Getting information from tensors

When dealing with tensors you probably want to be aware of the following attributes:
* Shape
* Rank
* Axis or dimension
* Size

In [ ]:
# Create a rank 4 tensor
rank_4_tensor = tf.zeros(shape=(2, 3, 4, 5))
rank_4_tensor

<tf.Tensor: shape=(2, 3, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
rank_4_tensor[0]

<tf.Tensor: shape=(3, 4, 5), dtype=float32, numpy=
array([[[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]], dtype=float32)>

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

(TensorShape([2, 3, 4, 5]), 4, <tf.Tensor: shape=(), dtype=int32, numpy=120>)

In [ ]:
2 * 3 * 4 * 5

120

In [ ]:
# Get various attributes of our tensor
print("Datatype of every element: ", rank_4_tensor.dtype)
print("Number of dimensions (rank): ", rank_4_tensor.ndim)
print("Shape of tensor: ", rank_4_tensor.shape)
print("Elements along the 0 axis: ", rank_4_tensor.shape[0])
print("Elements along the last axis: ", rank_4_tensor.shape[-1])
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor))
print("Total number of elements in our tensor: ", tf.size(rank_4_tensor).numpy())

Datatype of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
Shape of tensor:  (2, 3, 4, 5)
Elements along the 0 axis:  2
Elements along the last axis:  5
Total number of elements in our tensor:  tf.Tensor(120, shape=(), dtype=int32)
Total number of elements in our tensor:  120


### Indexing tensors

Tensors can be indexed just like Python lists.

In [ ]:
# Get the first 2 elements of each dimension

rank_4_tensor[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]]], dtype=float32)>

In [ ]:
# Get the first element from each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1, :]

<tf.Tensor: shape=(1, 1, 1, 5), dtype=float32, numpy=array([[[[0., 0., 0., 0., 0.]]]], dtype=float32)>

In [ ]:
# rank_4_tensor[:1, :1, :, :1]

In [ ]:
# rank_4_tensor[:1, :, :1, :1]

In [ ]:
# Create a rank 2 tensor
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([7, 4], dtype=int32)>

In [ ]:
# Add in extra dimension to our rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis] # ... = :, :
rank_3_tensor

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1)

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 3],
        [ 4]]], dtype=int32)>

In [ ]:
tf.expand_dims(rank_2_tensor, axis=1)

<tf.Tensor: shape=(2, 1, 2), dtype=int32, numpy=
array([[[10,  7]],

       [[ 3,  4]]], dtype=int32)>

In [ ]:
rank_2_tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

### Manipulating tensors (tensor operations)

**Basic operations**

`+`, `-`, `*`, `/`

In [ ]:
tensor = tf.constant([[10, 7],
                      [3, 4]])
tensor + 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 17],
       [13, 14]], dtype=int32)>

In [ ]:
# Original tensor is unchanged
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

In [ ]:
tensor * 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
tensor - 10

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0, -3],
       [-7, -6]], dtype=int32)>

In [ ]:
# We can use the TensorFlow built-in functions too
tf.multiply(tensor, 10)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  70],
       [ 30,  40]], dtype=int32)>

In [ ]:
tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4]], dtype=int32)>

**Matrix multiplication**


In [ ]:
print(tensor)
tf.matmul(tensor, tensor)

tf.Tensor(
[[10  7]
 [ 3  4]], shape=(2, 2), dtype=int32)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
tensor * tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[100,  49],
       [  9,  16]], dtype=int32)>

In [ ]:
tensor_1 = tf.constant([[1, 2, 5],
                        [7, 2, 1],
                        [3, 3, 3]])
tensor_2 = tf.constant([[3, 5],
                        [6, 7],
                        [1, 8]])
tf.matmul(tensor_1, tensor_2)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[20, 59],
       [34, 57],
       [30, 60]], dtype=int32)>

In [ ]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[121,  98],
       [ 42,  37]], dtype=int32)>

In [ ]:
tensor.shape

TensorShape([2, 2])

In [ ]:
# Create a tensor with shapes (3, 2)
X = tf.constant([[1, 2],
                 [3, 4],
                 [5, 6]])
# Create another (3, 2) tensor
Y = tf.constant([[7, 8],
                 [9, 10],
                 [11, 12]])
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
# tf.matmul(X, Y)

In [ ]:
tf.matmul(X, Y, transpose_b=True)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 7,  8,  9],
       [10, 11, 12]], dtype=int32)>

In [ ]:
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 58,  64],
       [139, 154]], dtype=int32)>

In [ ]:
tf.matmul(tf.transpose(X), Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

**The dot product**

In [ ]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[1, 2],
        [3, 4],
        [5, 6]], dtype=int32)>, <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32)>)

In [ ]:
tf.tensordot(tf.transpose(X), Y, axes=1)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 89,  98],
       [116, 128]], dtype=int32)>

In [ ]:
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 27,  30,  33],
       [ 61,  68,  75],
       [ 95, 106, 117]], dtype=int32)>

In [ ]:
tf.matmul(X, tf.transpose(Y))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 23,  29,  35],
       [ 53,  67,  81],
       [ 83, 105, 127]], dtype=int32)>

In [ ]:
print("Normal Y:")
print(Y, "\n")

print("Y reshaped to (2, 3):")
print(tf.reshape(Y, shape=(2, 3)), "\n")

print("Y transposed:")
print(tf.transpose(Y))

Normal Y:
tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32) 

Y reshaped to (2, 3):
tf.Tensor(
[[ 7  8  9]
 [10 11 12]], shape=(2, 3), dtype=int32) 

Y transposed:
tf.Tensor(
[[ 7  9 11]
 [ 8 10 12]], shape=(2, 3), dtype=int32)


### Changing the datatype of a tensor

In [ ]:
# Create a new tensor with default datatype (float32)
B = tf.constant([1.7, 7.4])
B.dtype

tf.float32

In [ ]:
C = tf.constant([7, 10])
C.dtype

tf.int32

In [ ]:
# Change from float32 to float16 (reduced precision)
D = tf.cast(B, dtype=tf.float16)
D

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([1.7, 7.4], dtype=float16)>

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 7., 10.], dtype=float32)>

In [ ]:
E_float16 = tf.cast(E, dtype=tf.float16)
E_float16

<tf.Tensor: shape=(2,), dtype=float16, numpy=array([ 7., 10.], dtype=float16)>

### Aggregating tensors

Aggregating tensors = condensing them from multiple values down to a smaller amount of values

In [ ]:
# Get the absolute values
D = tf.constant([-7, -10])
D

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ -7, -10], dtype=int32)>

In [ ]:
tf.abs(D)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([ 7, 10], dtype=int32)>

Let's go through the following forms of aggregation:
* Get the minimum
* Get the maximum
* Get the mean of a tensor
* Get the sum of a tensor

In [ ]:
# Create a random tensor with values between 0 and 100 of size 50
E = tf.constant(np.random.randint(0, 100, size=50))
E

<tf.Tensor: shape=(50,), dtype=int64, numpy=
array([30, 21, 65, 55, 15, 16, 65, 25, 59, 57, 84, 89, 86, 84, 98, 32, 27,
       48, 64, 41, 27, 69, 57, 47, 51, 53, 91, 98, 76,  1, 46, 14, 25,  9,
       63, 18,  9, 52, 25, 85,  4, 61, 61, 69, 76, 75, 46, 56, 57,  5])>

In [ ]:
tf.size(E), E.shape, E.ndim

(<tf.Tensor: shape=(), dtype=int32, numpy=50>, TensorShape([50]), 1)

In [ ]:
# Find the minimum
tf.reduce_min(E)

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [ ]:
np.min(E)

1

In [ ]:
# Find the maximum
tf.reduce_max(E)

<tf.Tensor: shape=(), dtype=int64, numpy=98>

In [ ]:
# Find the mean
tf.reduce_mean(E)

<tf.Tensor: shape=(), dtype=int64, numpy=49>

In [ ]:
# Find the sum
tf.reduce_sum(E)

<tf.Tensor: shape=(), dtype=int64, numpy=2487>

In [ ]:
# tf.math.reduce_variance(E)

TypeError: ignored

In [ ]:
# Find the variance
tf.math.reduce_variance(tf.cast(E, dtype=tf.float16))

<tf.Tensor: shape=(), dtype=float16, numpy=718.5>

In [ ]:
# tf.math.reduce_std(E)

TypeError: ignored

In [ ]:
# Find the standard deviation
tf.math.reduce_std(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=26.805826>

In [ ]:
import tensorflow_probability as tfp
tfp.stats.variance(E)

<tf.Tensor: shape=(), dtype=int64, numpy=719>

In [ ]:
tf.math.reduce_variance(tf.cast(E, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=718.55237>

**Find the positional maximum and minimum**


In [ ]:
# Find argmax
tf.math.argmax(E)

<tf.Tensor: shape=(), dtype=int64, numpy=14>

In [ ]:
E[14]

<tf.Tensor: shape=(), dtype=int64, numpy=98>

In [ ]:
# Find argmin
tf.math.argmin(E)

<tf.Tensor: shape=(), dtype=int64, numpy=29>

In [ ]:
tf.reduce_min(E) == E[29], E[29]

(<tf.Tensor: shape=(), dtype=bool, numpy=True>,
 <tf.Tensor: shape=(), dtype=int64, numpy=1>)

In [ ]:
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
       0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
       0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
       0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
       0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
       0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
       0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
       0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
       0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
       0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
      dtype=float32)>

In [ ]:
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=42>

In [ ]:
F[tf.argmax(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
tf.reduce_max(F)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9671384>

In [ ]:
tf.argmin(F)

<tf.Tensor: shape=(), dtype=int64, numpy=16>

In [ ]:
F[tf.argmin(F)]

<tf.Tensor: shape=(), dtype=float32, numpy=0.009463668>

### Squeezing a tensor (remove all single dimensions)

In [ ]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1, 1, 1, 1, 50))
G

<tf.Tensor: shape=(1, 1, 1, 1, 50), dtype=float32, numpy=
array([[[[[0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
           0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
           0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
           0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
           0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
           0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
           0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
           0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
           0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
           0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043]]]]],
      dtype=float32)>

In [ ]:
G.shape

TensorShape([1, 1, 1, 1, 50])

In [ ]:
G_squeezed = tf.squeeze(G)
G_squeezed, G_squeezed.shape

(<tf.Tensor: shape=(50,), dtype=float32, numpy=
 array([0.6645621 , 0.44100678, 0.3528825 , 0.46448255, 0.03366041,
        0.68467236, 0.74011743, 0.8724445 , 0.22632635, 0.22319686,
        0.3103881 , 0.7223358 , 0.13318717, 0.5480639 , 0.5746088 ,
        0.8996835 , 0.00946367, 0.5212307 , 0.6345445 , 0.1993283 ,
        0.72942245, 0.54583454, 0.10756552, 0.6767061 , 0.6602763 ,
        0.33695042, 0.60141766, 0.21062577, 0.8527372 , 0.44062173,
        0.9485276 , 0.23752594, 0.81179297, 0.5263394 , 0.494308  ,
        0.21612847, 0.8457197 , 0.8718841 , 0.3083862 , 0.6868038 ,
        0.23764038, 0.7817228 , 0.9671384 , 0.06870162, 0.79873943,
        0.66028714, 0.5871513 , 0.16461694, 0.7381023 , 0.32054043],
       dtype=float32)>, TensorShape([50]))

### One-hot encoding tensors

In [ ]:
# Create a list of indices
some_list = [0, 1, 2, 3] # could be red, green, blue, purple

tf.one_hot(some_list, depth=4)

<tf.Tensor: shape=(4, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)>

In [ ]:
# Specify custom values
tf.one_hot(some_list, depth=4, on_value="Here", off_value="Nope")

<tf.Tensor: shape=(4, 4), dtype=string, numpy=
array([[b'Here', b'Nope', b'Nope', b'Nope'],
       [b'Nope', b'Here', b'Nope', b'Nope'],
       [b'Nope', b'Nope', b'Here', b'Nope'],
       [b'Nope', b'Nope', b'Nope', b'Here']], dtype=object)>

### Squaring, log, square root

In [ ]:
H = tf.range(1, 10)
H

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
# Square it
tf.square(H)

<tf.Tensor: shape=(9,), dtype=int32, numpy=array([ 1,  4,  9, 16, 25, 36, 49, 64, 81], dtype=int32)>

In [ ]:
# Find the square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([1.       , 1.4142135, 1.7320508, 2.       , 2.236068 , 2.4494898,
       2.6457512, 2.828427 , 3.       ], dtype=float32)>

In [ ]:
# Find the log
tf.math.log(tf.cast(H, dtype=tf.float32))

<tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.       , 0.6931472, 1.0986123, 1.3862944, 1.609438 , 1.7917595,
       1.9459102, 2.0794415, 2.1972246], dtype=float32)>

### Tensors and NumPy

In [ ]:
# Create a tensor directly from NimPy array
J = tf.constant(np.array([3., 7., 10.]))
J

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 3.,  7., 10.])>

In [ ]:
np.array(J), type(np.array(J))

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
J.numpy(), type(J.numpy())

(array([ 3.,  7., 10.]), numpy.ndarray)

In [ ]:
J = tf.constant([3.])
J.numpy()[0]

3.0

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([3., 7., 10.]))
tensor_J = tf.constant([3., 7., 10.])
# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

(tf.float64, tf.float32)

### Finding access to GPUs

In [ ]:
import tensorflow as tf
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [ ]:
tf.config.list_logical_devices("GPU")

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [ ]:
!nvidia-smi

Tue Jan 24 19:43:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |    310MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

> **Note:** If you have acces to a CUDA-enabled GPU, Tensorflow will automatically use it whenever possible

## 🛠 Exercises

1. Create a vector, scalar, matrix and tensor with values of your choosing using `tf.constant()`.
2. Find the shape, rank and size of the tensors you created in 1.
3. Create two tensors containing random values between 0 and 1 with shape `[5, 300]`.
4. Multiply the two tensors you created in 3 using matrix multiplication.
5. Multiply the two tensors you created in 3 using dot product.
6. Create a tensor with random values between 0 and 1 with shape `[224, 224, 3]`.
7. Find the min and max values of the tensor you created in 6.
8. Created a tensor with random values of shape `[1, 224, 224, 3]` then squeeze it to change the shape to `[224, 224, 3]`.
9. Create a tensor with shape `[10]` using your own choice of values, then find the index which has the maximum value.
10. One-hot encode the tensor you created in 9.

### Exercise 1

In [ ]:
scalar = tf.constant(13)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=13>

In [ ]:
vector = tf.constant([26, 74, 100])
vector

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 26,  74, 100], dtype=int32)>

In [ ]:
matrix = tf.constant([[10, 18, 20],
                      [83, 10, 21]])
matrix

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[10, 18, 20],
       [83, 10, 21]], dtype=int32)>

In [ ]:
tensor = tf.constant([[[2.3, 5.6, 7.],
                       [21., 6, 13.3]],
                      [[324, 12, 94],
                       [10.7, 1.1, 1.3]],
                      [[19, 18, 17],
                       [16, 15, 14]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=float32, numpy=
array([[[  2.3,   5.6,   7. ],
        [ 21. ,   6. ,  13.3]],

       [[324. ,  12. ,  94. ],
        [ 10.7,   1.1,   1.3]],

       [[ 19. ,  18. ,  17. ],
        [ 16. ,  15. ,  14. ]]], dtype=float32)>

### Exercise 2

In [ ]:
scalar.shape, scalar.ndim, tf.size(scalar)

(TensorShape([]), 0, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [ ]:
vector.shape, vector.ndim, tf.size(vector)

(TensorShape([3]), 1, <tf.Tensor: shape=(), dtype=int32, numpy=3>)

In [ ]:
matrix.shape, matrix.ndim, tf.size(matrix)

(TensorShape([2, 3]), 2, <tf.Tensor: shape=(), dtype=int32, numpy=6>)

In [ ]:
tensor.shape, tensor.ndim, tf.size(tensor)

(TensorShape([3, 2, 3]), 3, <tf.Tensor: shape=(), dtype=int32, numpy=18>)

### Exercise 3

In [ ]:
tf.random.set_seed(13)
A = tf.random.uniform(shape=[5, 300], maxval=1)
B = tf.random.uniform(shape=[5, 300], maxval=1)

In [ ]:
A

<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
array([[0.5983684 , 0.07627809, 0.34005308, ..., 0.7458892 , 0.22718489,
        0.59905064],
       [0.62610304, 0.8146746 , 0.6315347 , ..., 0.6419549 , 0.24763274,
        0.01824832],
       [0.3870654 , 0.04646385, 0.25620794, ..., 0.08668923, 0.8227829 ,
        0.5537698 ],
       [0.58605766, 0.5132898 , 0.5560359 , ..., 0.8870833 , 0.4128605 ,
        0.17892277],
       [0.13492   , 0.19694388, 0.66055226, ..., 0.9596386 , 0.0360353 ,
        0.11811388]], dtype=float32)>

In [ ]:
B

<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
array([[0.53933334, 0.7998321 , 0.64869213, ..., 0.58930445, 0.06786788,
        0.83411944],
       [0.5909085 , 0.14960468, 0.819898  , ..., 0.83511245, 0.84207284,
        0.9188429 ],
       [0.19096875, 0.12252057, 0.367118  , ..., 0.04205072, 0.0052104 ,
        0.4683442 ],
       [0.7682309 , 0.6650634 , 0.9966041 , ..., 0.40728533, 0.7942357 ,
        0.61430943],
       [0.84820557, 0.58439493, 0.12084794, ..., 0.5354136 , 0.5738573 ,
        0.29079533]], dtype=float32)>

### Exercise 4

In [ ]:
A * B

<tf.Tensor: shape=(5, 300), dtype=float32, numpy=
array([[0.32272002, 0.06100966, 0.22058976, ..., 0.43955582, 0.01541856,
        0.49967977],
       [0.36996964, 0.12187913, 0.517794  , ..., 0.5361045 , 0.20852481,
        0.01676734],
       [0.0739174 , 0.00569278, 0.09405855, ..., 0.00364534, 0.00428703,
        0.2593549 ],
       [0.45022762, 0.34137025, 0.5541476 , ..., 0.36129603, 0.32790858,
        0.10991395],
       [0.1144399 , 0.11509301, 0.07982638, ..., 0.5138036 , 0.02067912,
        0.03434696]], dtype=float32)>

### Exercise 5

In [ ]:
tf.matmul(tf.transpose(A), B)

<tf.Tensor: shape=(300, 300), dtype=float32, numpy=
array([[1.3312745 , 1.0882977 , 1.6439686 , ..., 1.2026947 , 1.112744  ,
        1.6549358 ],
       [1.0927846 , 0.6450448 , 1.2698357 , ..., 1.0417516 , 1.2121251 ,
        1.2065333 ],
       [1.592957  , 1.1536791 , 1.4664164 , ..., 1.318705  , 1.3768982 ,
        1.5175831 ],
       ...,
       [2.2936301 , 1.8540213 , 2.0420563 , ..., 1.8544053 , 1.8468952 ,
        2.076619  ],
       [0.77372   , 0.6152019 , 1.0682783 , ..., 0.5627267 , 0.5768181 ,
        1.0664835 ],
       [0.67726266, 0.7377383 , 0.799449  , ..., 0.5276614 , 0.26879543,
        0.9200629 ]], dtype=float32)>

In [ ]:
tf.matmul(A, tf.transpose(B))

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[78.55892 , 74.90392 , 76.32059 , 74.04147 , 77.159775],
       [76.70238 , 72.94185 , 73.8091  , 73.22002 , 75.83854 ],
       [73.413994, 71.14287 , 69.15992 , 67.73338 , 71.9556  ],
       [73.651886, 73.330574, 71.45819 , 70.031456, 70.42944 ],
       [73.593   , 69.29292 , 74.3016  , 68.19471 , 72.06778 ]],
      dtype=float32)>

### Exercise 6

In [ ]:
tf.random.set_seed(13)
C = tf.random.uniform(shape=[224, 224, 3], maxval=1)
C

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[0.5983684 , 0.07627809, 0.34005308],
        [0.9105623 , 0.65763414, 0.2534064 ],
        [0.25697923, 0.9059994 , 0.4093889 ],
        ...,
        [0.5289855 , 0.6754769 , 0.7332182 ],
        [0.07460606, 0.8198025 , 0.43353236],
        [0.10257089, 0.33321774, 0.10659897]],

       [[0.05175698, 0.66115093, 0.6400194 ],
        [0.38941896, 0.666448  , 0.7997081 ],
        [0.02462029, 0.03575015, 0.6462735 ],
        ...,
        [0.50282025, 0.79770243, 0.17889082],
        [0.8008561 , 0.10912907, 0.44630527],
        [0.9535289 , 0.08510852, 0.13136816]],

       [[0.51533043, 0.09120989, 0.6249205 ],
        [0.87484837, 0.6831188 , 0.24161756],
        [0.14628375, 0.96835756, 0.6252203 ],
        ...,
        [0.8559567 , 0.9372302 , 0.74666476],
        [0.36533546, 0.4592687 , 0.6805129 ],
        [0.97679853, 0.9350158 , 0.98095644]],

       ...,

       [[0.5455816 , 0.6884854 , 0.48416424],
        [0.49

### Exercise 7

In [ ]:
tf.reduce_max(C)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9999995>

In [ ]:
tf.reduce_min(C)

<tf.Tensor: shape=(), dtype=float32, numpy=1.66893e-06>

### Exercise 8

In [ ]:
tf.random.set_seed(13)
D = tf.random.normal(shape=[1, 224, 224, 3])
D

<tf.Tensor: shape=(1, 224, 224, 3), dtype=float32, numpy=
array([[[[ 4.6733698e-01,  8.9927393e-01, -7.8262424e-01],
         [ 1.2429025e+00,  9.1533005e-01, -1.9593909e-02],
         [-9.1800916e-01,  1.3692257e+00, -1.3188699e+00],
         ...,
         [-2.3625762e+00, -8.8089770e-01, -9.3230255e-02],
         [-2.0627525e+00,  9.6753263e-01,  7.7674079e-01],
         [ 1.0335625e+00,  9.2038077e-01,  1.1622467e+00]],

        [[-2.0641296e+00, -1.2890934e+00,  6.0484672e-01],
         [-7.2572309e-01, -8.5729772e-01,  2.7681586e-01],
         [ 6.0626256e-01,  2.6534531e+00, -3.6460876e-01],
         ...,
         [-2.7048716e-01,  6.0634047e-01,  2.9050055e-01],
         [ 4.2199126e-01,  5.1582110e-01, -3.6564490e-01],
         [ 1.2164736e+00,  1.6311911e+00,  1.5056133e+00]],

        [[ 6.2436390e-01,  9.6750379e-01, -6.8631393e-01],
         [ 6.8500727e-01,  8.7181461e-01,  4.5959588e-02],
         [-3.8725740e-01,  1.9220941e+00,  6.5909553e-01],
         ...,
         [ 

In [ ]:
E = tf.squeeze(D)
E

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 4.6733698e-01,  8.9927393e-01, -7.8262424e-01],
        [ 1.2429025e+00,  9.1533005e-01, -1.9593909e-02],
        [-9.1800916e-01,  1.3692257e+00, -1.3188699e+00],
        ...,
        [-2.3625762e+00, -8.8089770e-01, -9.3230255e-02],
        [-2.0627525e+00,  9.6753263e-01,  7.7674079e-01],
        [ 1.0335625e+00,  9.2038077e-01,  1.1622467e+00]],

       [[-2.0641296e+00, -1.2890934e+00,  6.0484672e-01],
        [-7.2572309e-01, -8.5729772e-01,  2.7681586e-01],
        [ 6.0626256e-01,  2.6534531e+00, -3.6460876e-01],
        ...,
        [-2.7048716e-01,  6.0634047e-01,  2.9050055e-01],
        [ 4.2199126e-01,  5.1582110e-01, -3.6564490e-01],
        [ 1.2164736e+00,  1.6311911e+00,  1.5056133e+00]],

       [[ 6.2436390e-01,  9.6750379e-01, -6.8631393e-01],
        [ 6.8500727e-01,  8.7181461e-01,  4.5959588e-02],
        [-3.8725740e-01,  1.9220941e+00,  6.5909553e-01],
        ...,
        [ 1.3167341e+00, -3.5999

### Exercise 9

In [6]:
F = tf.constant([9, 5.6, 21, 3.219, 7.9999, 6, 12, 25.32, 0, 17.19])
F

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([ 9.    ,  5.6   , 21.    ,  3.219 ,  7.9999,  6.    , 12.    ,
       25.32  ,  0.    , 17.19  ], dtype=float32)>

In [7]:
tf.argmax(F)

<tf.Tensor: shape=(), dtype=int64, numpy=7>

### Exercise 10

In [8]:
G = tf.one_hot(tf.cast(F, dtype=tf.int32), depth=F.numpy().max()+1)
G

<tf.Tensor: shape=(10, 26), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.,